# <span style="color:blue">EVCO Workshop 7: Competitive Coevolution</span>

Simon O'Keefe: simon.okeefe@york.ac.uk

Danny Roberts: danny.roberts@york.ac.uk

Tianda Sun: tianda.sun@york.ac.uk

## <span style="color:#0073e6">Learning objectives</span>

- To implement a simple competitive coevolutionary algorithm using DEAP
- To understand how the populations interact and the impact of changing parameters in each population
- Implement a common version of fitness sharing (diversity measure)

# <span style="color:blue">Practical Instructions</span>

This task takes a toy game to explore competitive evolution in a simplistic domain. The game is called the Matching Game and is a coevolutionary version of the max-ones game. If you want to read more about a coevolutionary algorithm implementing this game, then you can read the paper on the VLE (matchingGame.pdf).

In this simple immune system-inspired game, there are two populations: parasites and hosts. Each population is represented by a bit-string. Parasites are rewarded for their number of binary genes that match the corresponding binary genes of hosts, but hosts are rewarded for corresponding genes mismatching with parasites. Games of this type often result in coevolutionary cycling, because hosts chase parasites through solution space.

**Fitness function:** Not all genes are always involved in this matching game. For parasites with many 1-alleles, the matching game tends to involve only those genes at which the parasite possesses 1-alleles. For parasites with many 0-alleles, the game tends to involve only those genes at which the parasite possesses 0-alleles. Whether 1-allele gene or 0-allele gene are involved is determined probabilistically. The probability, p, of a game involving matching 1-alleles increases with the total number of 1-alleles, x, that the parasite has such that:

<img src="equ.png" alt="equation" width=250>

Otherwise the game matches based on only 0-alleles for that parasite and its hpst opponents.

Once the game has been decided, the fitness of the host is the number of genes that it mismatches for each individual, and the fitness of the parasite is the number of genes that match for each individual.

**If you want to start simpler, you can implement the game with a simple evaluation function based simply on the number of genes matched (for the parasite) or unmatched (for the host), and then move on to this more complicated version.**

# <span style="color:blue">Exercise 1: Implement the coevolutionary problem</span>

Your task is to implement this game in DEAP. DEAP does not implement parasitic coevolution directly. However, parasitic evolution is simply two evolutionary algorithms running simultaneously, with the fitness of each population linked. So you can implement this using your current knowledge of DEAP. Implement the problem and try to understand the results.

**Implementation Details**

I suggest using the max-ones code you developed in the first practical as a starting point.
- Use two distinct populations (species) of size 50 each (hosts / parasites)
- Individuals in each population consist of binary strings containing 100 bits, initialised at random at generation 0.
- In each generation, members of the host population are selected to play a set of pair-wise contests against a random sample of 5 opponents from the parasite population.
- Each individual has a small probability of a bit-flip mutation with a probability per gene of 0.05 (indpb=0.05) for each population
- Use tournament selection with size 6
- Run for 150 generations to start with
- Unlike standard evolution, you should **not** check to see if an individual has been modified (e.g. remove ‘invalid_ind’ and ‘del mutant.fitness.values’) before assessing their fitness, because relative fitness can change even if they have not been modified. Thus, you always need to reassess fitness.

**Things to do:**
- To explore the coevolution of the hosts and parasites, plot the average number of 1s in each generation, for each population alongside each other on a graph.
- Try giving a high mutation rate (e.g. indpb=0.1) to the parasite, and lower to the host (e.g. indpb=0.01). How does this change the coevolutionary dynamic?
- Now reverse those mutation rates, so that the host has a mutational advantage. How does this change the coevolutionary dynamic?

## <span style="color:#0073e6">Live plotting</span>

As an optional extra, we could plot progress as the code runs. You do not have to do this, and could chose to save progress and plot later too. If you choose to do it, make sure that you import 'display' from IPython first.

In [5]:
from IPython import display
from deap import base, creator, tools, algorithms
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [1]:
import random

from deap import base, creator, tools

# define the fitness and individual type using the DEAP library's creator module
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# define the toolbox with mutation and selection operators using the DEAP library's base module
toolbox = base.Toolbox()

def generate_individual(length):
    return [random.randint(0, 1) for _ in range(length)]

# register the generate individual function as a method of the DEAP toolbox
toolbox.register("individual", generate_individual, length=10)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate_fitness(individual):
    return sum(individual),

toolbox.register("evaluate", evaluate_fitness)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("fitness")


In [2]:
population_size = 100
num_generations = 50
crossover_probability = 0.5
mutation_probability = 0.2

random.seed(42)

population = toolbox.population(n=population_size)

for individual in population:
    individual.fitness.values = toolbox.evaluate(individual)

for generation in range(num_generations):
    offspring = [toolbox.clone(ind) for ind in population]

    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < crossover_probability:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < mutation_probability:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    invalid_individuals = [ind for ind in offspring if not ind.fitness.valid]
    fitness_values = [toolbox.evaluate(ind) for ind in invalid_individuals]
    for individual, fitness_value in zip(invalid_individuals, fitness_values):
        individual.fitness.values = fitness_value

    population[:] = offspring

    fits = [ind.fitness.values[0] for ind in population]
    print(f"Generation {generation}: Max fitness = {max(fits)}")

best_individual = tools.selBest(population, k=1)[0]
print(f"\nBest individual: {best_individual}\nFitness: {best_individual.fitness.values[0]}")


AttributeError: 'list' object has no attribute 'fitness'

At the end of each generation, within the loop, you can now plot the number of ones for each individual in each species using the code:

In [3]:
gen = [g] * POPSIZE
host = [sum(x) for x in popHost]
parasite = [sum(x) for x in popParasite]
plt.scatter( gen, host, color='red', s=15, alpha=0.05)
plt.scatter( gen, parasite, color='blue', s=15, alpha=0.05)
display.display(plt.gcf())
display.clear_output(wait=True)

NameError: name 'g' is not defined

Where g is the current generation, POPSIZE is your population size (assuming equal population sizes for each species here), popHost is your host population and popParasite is your parasite population.

# <span style="color:blue">Exercise 2: Diversity measures and general strategies</span>

You can see that short-term strategies and cycling can occur a lot. This is simply a property of this problem. However, transitivity can be a real issue when it comes to other problems. As you have seen in the lectures, diversity measures can help with not only this, but with evolvability in general (and are particularly important for coevolution and multi-objective evolution).

Fitness sharing is a commonly used diversity method that we will implement. Although it will not make much difference for this problem, the intention here is to understand how to implement it for this problem.

**Watch the short pre-recorded mini-lecture on fitness sharing (under workshop materials)** where I go into more detail on the method. Here is the pseudocode for how you would implement it here. To calculate the shared fitness for individual i:

```
alpha = 0.5 
sigma = 5

res = 0
for each individual j in the population 
    calculate the genotype distance from i to j
    if distance < sigma
        res += 1 – (distance/sigma)**alpha
fitness = (original_fitness / res),    
```

For our binary genomes, we can calculate the difference between genomes using the Hamming distance, which simply returns the number of positions where the two individuals differ. Below is a Python implementation of Hamming distance between two genotypes (s1 and s2):

In [2]:
def hamming(s1, s2):
    assert len(s1) == len(s2)
    return sum(c1 != c2 for c1, c2 in zip(s1, s2))

**Your task:** Implement fitness sharing for both populations.